# Argentina documents from lens from 2013

https://link.lens.org/Kr7ZrPzLlJf: Downloaded JSON files [here](https://drive.google.com/drive/folders/1tnLxHd4Jz2s35OdFlnqXRFv4iUEqZ76h?usp=sharing)

[Google Scholar](http://scholar.google.com/scholar_lookup?&title=WIMP+dark+matter+as+radiative+neutrino+mass+messenger&journal=JHEP&volume=10&publication_year=2013&author=Hirsch,M&author=Lineros,RA&author=Morisi,S&author=Palacio,J&author=Rojas,N&author=Valle,JWF') example 

## Prepare data collection

### With  Pandas
Requieres enough RAM

Get the lens.org JSON files

In [119]:
import pandas as pd

In [120]:
def read_and_simplify_json(file):
    ar=pd.read_json(file)

    ar['authors']=ar['authors'].apply(lambda l: ' and '.join( 
                    [f'{d.get("last_name")}, {d.get("initials")}' for d in l][:3]
                          ))

    ar['source']=ar['source'].apply(lambda d: d.get('title_full') if isinstance(d,dict) else '')

    ar['external_ids']=ar['external_ids'].apply(lambda l: [d.get('value') for d in l if d.get('type')=='doi'] if isinstance(l,list) else []).str[0]

    ar=ar.rename({'authors':'author','source':'journal','external_ids':'doi','year_published':'year','lens_id':'article_id'
                 },axis='columns')[
        ['article_id','title','author','journal','year','volume','issue','doi'] 
        ].fillna('')
    return ar

In [79]:
ar=pd.DataFrame()
files=['lens_ar_2013.json','lens_ar_2014_2016.json','lens_ar_2017_2018.json','lens_ar_2019.json']

In [80]:
for f in files:
    ar=ar.append( read_and_simplify_json(f) ).reset_index(drop=True)

In [81]:
ar['publisher']=''
ar['country']=''
ar["pages"]=''
ar["abstract"]=''
ar["language"]=''

In [ ]:
ar.to_json('argentina.json',orient='records')

```bash
mongoimport --db la --collection data --file /scratch/restrepo/argentina.json --jsonArray
hunabku_server
moai_gslookup --hunabku_server http://10.0.0.8:8080 --proxy_api 12894ec7c83f91ffcdff8fb116e75906 --max_threads 50 --db la --max_tries 2
```

In [43]:
ar=pd.read_json('argentina.json')

### With Mongodb
Only disc

In [64]:
%%bash
RELOAD=0
if [ $RELOAD == 1 ];then
    for f in lens_ar_2013.json lens_ar_2014_2016.json lens_ar_2017_2018.json lens_ar_2019.json ;do 
        mongoimport --db la --collection lens --file $f --jsonArray
    done
fi

In [79]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import sys

In [20]:
client = MongoClient()

In [21]:
client = MongoClient('localhost', 27017)

In [66]:
#client.drop_database('la')

In [68]:
client.list_database_names()

['la', 'local']

In [69]:
db = client['la']

In [70]:
db.list_collection_names()

['stage_cites',
 'stage_invalid',
 'data',
 'stage',
 'quarantine',
 'stage_cites_invalid',
 'lens',
 'cache_cites',
 'gslookup_not_found']

In [71]:
ln=db.get_collection('lens')

In [72]:
ln.count_documents({})

136326

In [80]:
REBUILD=False
if not REBUILD:
    dt=db.get_collection('data')
    sys.exit('Already loaded')
else:
    dt=db.get_collection('data')

for p in ln.find():
    dd={}
    l=[]
    for d in p['authors'][:3]:
        l.append(f'{d.get("last_name")}, {d.get("initials")}')
    dd['author']=' and '.join(l)
    if isinstance(p.get('source'),dict):
        dd['journal']=p['source'].get('title_full')
    else:
        dd['journal']=''
    for d in p['external_ids']:
        if d.get('type')=='doi':
            doi=d.get('value')
        else:
            doi=''
    dd['doi']=doi
    dd["pages"]=p.get('start_page')            

    dd['title']=p['title']
    dd['volume']=p.get('volumne')
    dd['issue']=p.get('issue')     
    
    dd['year']=p['year_published']                     
    dd['article_id']=p['lens_id']                     
    dd['publisher']=''
    dd['country']=''
    dd["abstract"]=''
    dd["language"]=''
    dt.insert_one(dd)
    #break

SystemExit: Already loaded

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [74]:
dt.count_documents({})

133853

In [75]:
#dt.drop()

### Add the lens collection `'_id'` to data as `'pid'` by searching with `'title'`

In [76]:
ln.count_documents({})

136326

In [77]:
dt.count_documents({})

133853

In [34]:
import time

In [81]:
REBUILD=False
if not REBUILD:
    sys.exit('Already fixed')
#See https://stackoverflow.com/a/24200795/2268280
cursor = ln.find(
     no_cursor_timeout=True
)

s=time.time()
for d in cursor:
    title=d.get('title')
    pid=str(d.get('_id'))
    if title and pid:
        kk=dt.update_one( {"title":title}, {"$set" : {"pid":pid}}   )
    #break
cursor.close()
print(time.time()-s)

SystemExit: Already fixed

In [82]:
i=0
iend=55
for d in dt.find():
    if i==iend:
        break
    i=i+1

In [83]:
d['pid'],d['title']

('60e63d00cdd7c50a878a7984',
 'Parasitism and physiological trade-offs in stressed capybaras.')

In [84]:
dd=list(ln.find({'_id':ObjectId(d['pid'])}))[0]

In [85]:
dd['title']

'Parasitism and physiological trade-offs in stressed capybaras.'

### Check de collection with Pandas

In [86]:
import pandas as pd

In [87]:
ar=pd.DataFrame(list(dt.find()))
ar[:3]

,_id,title,author,journal,year,volume,issue,doi,publisher,country,pages,abstract,article_id,language,pid
0,60d6957a70d175acd8cb1340,Low vitamin D concentrations among indigenous ...,"Hirschler, V and Maccallini, G and Molinari, C",Pediatric diabetes,2013,14,3,,,,,,,,60e63d00cdd7c50a878a794f
1,60d6957a70d175acd8cb1341,Investigaciones sobre género y determinación p...,"Tajer, D and Reid, G and Gaba, M",Revista Argentina de Cardiología,2013,81,4,10.7775/rac.es.v81.i4.280,,,,,,,60e63d00cdd7c50a878a7949
2,60d6957a70d175acd8cb1342,Duplex Stainless Steels - Low‐Cycle Fatigue at...,"Alvarez-Armas, I",Duplex Stainless Steels,2013,,,10.1002/9781118557990.ch6,,,,,,,60e63d00cdd7c50a878a7947


## Obtain the cites
`--proxy_api` key from https://dashboard.scraperapi.com/dashboard

```bash
hunabku_server #in and indepentdent, nohup, other terminal
```

### Get de Google Scholar info for each document
Including `'cid'` and bibtex. The info is stores in the stage collection

```bash
moai_gslookup --hunabku_server http://10.0.0.8:8080 --proxy_api XXXXXX --max_threads 50 --db la --max_tries 2
```

Check the proxies

In [89]:
st=db.get_collection('stage')
st.count_documents({})

125987

In [90]:
s=st.find_one()
cid=s['cid']
(s['title'],s['author'],s['journal'])

('Low vitamin D concentrations among indigenous Argentinean children living at high altitudes',
 'Hirschler, Valeria and Maccallini, Gustavo and Molinari, Claudia and Aranda, Claudio and San Antonio de los Cobres Study Group',
 'Pediatric diabetes')

Check with:

In [91]:
print( f'https://scholar.google.com/scholar?q=info:{cid}:scholar.google.com' )

https://scholar.google.com/scholar?q=info:bCwnnx4CbTsJ:scholar.google.com


In [92]:
ar=pd.DataFrame(list(st.find()))

In [93]:
ar.cites.fillna(0).sum()

1277822.0

### Creates de chache
```bash
moai_gscites  --hunabku_server http://127.0.1.1:8080 --proxy_apikey xxxxxx --db la --create_cache

```

### Download metadata info for each cite
Creates 
```bash
moai_gscites  --hunabku_server http://127.0.1.1:8080 --proxy_apikey xxxxxx --db la --max_threads 5  --max_tries 2
```

In [94]:
sc=db.get_collection('stage_cites')
sc.count_documents({})

1076343

In [147]:
sce=sc.find({'pid':str(s['_id'])})

In [148]:
ar=pd.DataFrame(list(sce))
ar[:1]

,_id,pid,cite
0,60dae3041d38555bfdc170a2,60d6957a70d175acd8cb1340,{'journal': 'American journal of hypertension'...


In [125]:
cites=s['cites_link'].split('cites=')[1].split('&')[0]

In [ ]:
ar=pd.DataFrame( ar['cite'].to_list())

To check with

In [126]:
f'https://scholar.google.com/scholar?cites={cites}'

'https://scholar.google.com/scholar?cites=4282081151241628780'

In [116]:
ar[['author','title','journal','bibtex']]

,author,title,journal,bibtex
0,"Tomaino, Katherine and Romero, Karina M and Ro...",Association between serum 25-hydroxy vitamin D...,American journal of hypertension,"@article{tomaino2015association,\n title={Ass..."
1,"Hirschler, Valeria and Maccallini, Gustavo and...",Improvement in high-density lipoprotein choles...,Hormone research in paediatrics,"@article{hirschler2013improvement,\n title={I..."
2,"Hirschler, V",Cardiometabolic risk factors in native populat...,International journal of clinical practice,"@article{hirschler2016cardiometabolic,\n titl..."
3,"Orces, Carlos H",Vitamin D status among older adults residing i...,The Scientific World Journal,"@article{orces2015vitamin,\n title={Vitamin D..."
4,"Farrokhyar, Forough and Tabasinejad, Rasam and...",Prevalence of vitamin D inadequacy in athletes...,Sports medicine,"@article{farrokhyar2015prevalence,\n title={P..."
5,"Kelishadi, Roya and Ardalan, Gelayol and Motla...",National report on the association of serum vi...,Nutrition,"@article{kelishadi2014national,\n title={Nati..."
6,"Skutsch, Margaret and Dobler, Carlos and McCal...",The association of UV with rates of COVID-19 t...,medRxiv,"@article{skutsch2020association,\n title={The..."
7,"Gassmann, Max and Mairbäurl, Heimo and Livshit...",The increase in hemoglobin concentration with ...,Annals of the New York Academy of Sciences,"@article{gassmann2019increase,\n title={The i..."
8,", and ,",D ...,. .,"@article{ 2016 ,\n title={ ..."
9,"OUASTI, Asmaa and AGUILI, Zeyneb",LStatut De LStatut De LStatut De La Vitamine D...,,"@phdthesis{ouastilstatut,\n title={LStatut De..."


Convert `ObjectId` to `str`

In [136]:
sid=str(s['_id'])

In [145]:
pd.DataFrame( list( st.find({'_id':ObjectId( sid )}) ) )[['article_id','title','author','journal','cites']]

,article_id,title,author,journal,cites
0,,Low vitamin D concentrations among indigenous ...,"Hirschler, Valeria and Maccallini, Gustavo and...",Pediatric diabetes,40


## Check the API

In [151]:
import requests
import getpass

In [152]:
api=getpass.getpass('API key')

API key ································


In [153]:
rr=requests.get(f"http://api.scraperapi.com/account?api_key={api}")

In [154]:
rr.json()

{'concurrencyLimit': 5,
 'concurrentRequests': 0,
 'failedRequestCount': 53,
 'requestCount': 925,
 'requestLimit': '1000'}

## Dump

```bash
mongodump -d la
#To restore in a new mongo without the la database:
mongorestore ./dump/
```

Google Scholar Profile
* cites can be obtained directly
* Click on title and in the open up change the "related:" by "info:"

## Quality check

In [ ]:
ar=pd.DataFrame()
cursor = sc.find(
     no_cursor_timeout=True
)

s=time.time()
i=0
for d in cursor:
    ar=ar.append( {'title': d.get('cite').get('title'),'cid':d.get('cite').get('cid')},ignore_index=True )
    i+=1
    if i%10000==0:
        print(i,end='\r')
    #break
cursor.close()
print(time.time()-s)